# 理論2-1層：最急降下法

- 物語：近くで一番良いものを進める限り選ぶ
- なぜ：近くというのは重要な概念である
- 特徴: 局所解、超高速最適化
- 注意: すぐ行き詰まる
- 似た物：目先の利益、近くが微分値だとニュートンラプソン法となる

## シミュレーション

計算手順

- 計算できる正しさがある(評価関数)
- 無作為の「状態」から始まる
- a. 今の状態の近くに次の状態がある。近くから評価の一番高い者を選ぶ。無ければ終わり。
- aを繰り返す

理論1層と同様に巡回セールスマン問題

## 手順

- 上の理論から想像しましょう。どうなるでしょうか
    - 下にプログラムがありますので詳細を確認してもよいかもしれません
- VsCode: Run All
- web版: 上部メニュー ＞ Kernel ＞ Restart ＆ Run All を押す
- 画像を比較する。総距離を比較する。考察する
- 興味があれば、パラメータ・プログラムを改変して、手順を再度行います

In [ ]:
# 言語 python3

import random
import numpy as np
import math
from PIL import Image, ImageDraw

WIDTH = 300  # 図の幅
HEIGHT = 300 # 図の高さ
REPORT_TIMES = 100 # 情報を出力する回数
REPORT_ITER = 1 # 情報を出力するループ回数
MAX_ITER = REPORT_TIMES * REPORT_ITER

###################################################
# トラベラーズセールスマン問題
class TSP:
    # 定数
    NUM = 12 # 拠点の数
    def __init__(self):
        self.order = randomArray(self.NUM) # 巡る順番(状態)
        self.distCache = makeArray2D(self.NUM, self.NUM, 0) # 点から点への距離
        vertexPos = []
        for i in range(self.NUM):
            vertexPos.append([0, 0])
            vertexPos[i][0] = random.random()
            vertexPos[i][1] = random.random()
        self.vertexPos = np.asarray(vertexPos, dtype=np.float64) # 拠点の位置
        self.calcDistanceCache()

    # 各点の距離
    def calcDistanceCache(self):
        for i in range(self.order.size):
            for j in range(self.order.size):
                if(i == j):
                    self.distCache[i][j] = 0
                else:
                    self.distCache[i][j] = squareRoot(self.vertexPos[i][0], self.vertexPos[j][0], self.vertexPos[i][1], self.vertexPos[j][1])

    # 総距離
    def calcTotalDistance(self, order):
        total = 0
        for i in range(order.size - 1):
            total += self.distCache[order[i]][order[i+1]]
        total += self.distCache[order[order.size - 1]][order[0]]
        return total
    
    # interface
    def calcScore(self, state):
        return self.calcTotalDistance(state)
    
    def getState(self):
        return self.order
    
    def setState(self, state):
        self.order = state
    
    def draw(self):
        im = Image.new('RGB', (WIDTH, HEIGHT), (192, 192, 192))
        drawObj = ImageDraw.Draw(im)
        for i in range(1, self.order.size):
            drawLine(drawObj, self.vertexPos[self.order[i-1]][0], self.vertexPos[self.order[i-1]][1], self.vertexPos[self.order[i]][0], self.vertexPos[self.order[i]][1])
        drawLine(drawObj, self.vertexPos[self.order[self.order.size-1]][0], self.vertexPos[self.order[self.order.size-1]][1], self.vertexPos[self.order[0]][0], self.vertexPos[self.order[0]][1])
        im.show()
        
###################################################
# 最急降下法
class GD:
    # 初期化
    def __init__(self, problem):
        self.iteration = 0  # ループした回数
        self.score = 0
        self.problem = problem
        self.n = self.problem.getState().size

    # 全体の流れ
    def solve(self):
        isSolved = self.calcNext()
        self.score = self.problem.calcScore(self.problem.getState())
        self.iteration += 1
        return isSolved

    # 近傍から次の状態へ遷移する
    def calcNext(self):
        for i in range(0, self.n - 1):
            for j in range(i + 1, self.n - 1):
                scoreBefore = self.problem.calcScore(self.problem.getState())
                nextOrder = swap(self.problem.getState().copy(), i, j)
                scoreAfter = self.problem.calcScore(nextOrder)
                subScore = scoreAfter - scoreBefore
                if(subScore < 0):
                    self.problem.setState(nextOrder)
                    return False
        return True

###################################################
# 出力
def report(solver):
    print("ーーーーーーーーーーーーー")
    print("繰返回数 : "+ str(solver.iteration))
    print("距離"+  str(solver.score))
    solver.problem.draw()

###################################################

def drawLine(drawObj, beforeX, beforeY, afterX, afterY):
    drawObj.line([(int(beforeX* WIDTH), int(beforeY*HEIGHT)), (int(afterX*WIDTH), int(afterY*HEIGHT))], fill="blue", width=1)

# 二次元配列作成
def makeArray2D(n, m, value):
    ret = []
    for i in range(n):
        ret.append(list(range(m)))
        for j in range(m):
            ret[i][j] = value
    return np.asarray(ret, dtype=np.float64) # 高速化のため配列

def twice(x):
    return x * x

def squareRoot(ax, bx, ay, by):
    return math.sqrt(twice(ax - bx) + twice(ay - by))

def randomArray(n):
    order = list(range(n)) 
    random.shuffle(order)
    return np.asarray(order)

###################################################
random.seed() # 乱数初期化
solver = GD(TSP())     # 初期化
solver.problem.draw()
for i in range(REPORT_TIMES):
    for j in range(REPORT_ITER):
        if solver.solve():
            break
    else:
        report(solver)
        continue
    break
print("終了")

### 考察（要検証）


### 閃き方法 (要検証)

- 普通に思いつく

### 生活の知恵　（要検証）



# 人工ニューラルネットワーク(途中)

- 物語：人の脳は経験と結果から覚える
- なぜ：近傍の応用で重要であるため（最急降下法系ロジックが役に立つという事実）
- 特徴
    - 汎化（さまざまな局面に対応できる）
    - 過剰適応（同じことを何回も行うと汎化が弱くなる）
    - 初期値に強く依存する (近傍に答えが無くてはいけない)
    - 大量の教師データ
    - 膨大な計算
    - 最適化結果を覚える
- 注意: 基礎であり、この理論だけでは役に立たない


## シミュレーション

### 計算手順

- データ上の入力と出力の対応がたくさんある(人間の経験：できごとと結果：評価)(評価関数)
- 特定の範囲内の正規分布の膨大な中間層から始まる (行列：状態)
- 1. データを入力から出力まで流す（行列積）。結果が0,1になるように対応させる
- 2. 計算結果と教師データの差分(偏差)を「入力から出力までの逆関数（出力から入力までの関数）」使って、誤差が0になるようにニュートンラプソン法をして、中間層を最適化する。
- 違うデータでa.から繰り返す
- 全データ使ったら、最初のデータから
- 適当にやめると中間層は学習済み

### 画像の判別シミュレーション

- データについて
    - 数字の画像を幅16ビット、高さ16ビット (白黒)画像を32枚用意する
    - 答えは0～3の4種類。それぞれ 0 または 1 との4要素配列

- 学習手順について
    - 画像を読み込む
    - 画像を1列 256ビットに変形する(16 * 16) x\[32 | num \] \[ 256 | image bit \]
    - 答えをセット ans\[32 | num \]\[ 4| ans \]
    - 中間層を初期化(標準偏差1平均0とかの乱数) w1\[256\]\[4096\] w2\[4096\]\[4\]
    - A: x = 1 ... 32
    - x 内積 w1 内積 w2 = y (numpy.dot：内積)
    - sigmoid関数(連続)で0,1にする（連続の逆関数があるので、逆関数の微分可能）
    - ans - y の最小二乗法とか（結果と答えの差の安定した評価。4つの答えの間違いが大きいのが一個でもあると低評価）
    - 手抜きで自動微分を使ってw1, w2を更新(最適化の一歩)します
    - つぎのx で Aから繰り返す
    - 終わったら x = 1 で Aから何回か繰り返す
    - 途中経過から判断し終了

- 推論について
    - 学習済み nn を用意
    - 画像に対して推論関数を呼ぶ(x 内積 w1 内積 w2 = y)
    - y がそのまま答え

### 手順

- 上の理論から想像しましょう。どうなるでしょうか
    - 下にプログラムがありますので詳細を確認してもよいかもしれません
- VsCode: Run All
- web版: 上部メニュー ＞ Kernel ＞ Restart ＆ Run All を押す
- 精度は良くないでしょうけど、雰囲気だけでもみてください
- 興味があれば、パラメータ・プログラムを改変して、手順を再度行います


In [ ]:
# データ読み込み(書きかけ)

images = [
    [ # 2
        [0,0,0,0,0,0,0,0],
        [0,0,0,1,1,0,0,0],
        [0,0,1,0,0,1,0,0],
        [0,1,0,0,0,1,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,1,0,0,0,0],
        [0,0,1,0,0,0,0,0],
        [0,0,1,1,1,1,1,0],
    ],
    [ # 1
        [0,0,0,0,0,0,0,0],
        [0,0,0,1,1,0,0,0],
        [0,0,1,0,1,0,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,1,1,1,0,0],
        [0,0,0,0,0,0,0,0],
    ],
    [ # 0
        [0,0,0,0,0,0,0,0],
        [0,0,1,1,1,1,0,0],
        [0,1,0,0,0,0,1,0],
        [0,1,0,0,0,0,1,0],
        [0,1,0,0,0,0,1,0],
        [0,1,0,0,0,0,1,0],
        [0,0,1,1,1,1,0,0],
        [0,0,0,0,0,0,0,0],
    ],
    [ # 3
        [0,0,0,0,0,0,0,0],
        [0,0,1,1,1,1,0,0],
        [0,0,0,0,0,0,1,0],
        [0,0,0,0,0,1,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,0,0,1,0,0],
        [0,0,1,0,0,0,1,0],
        [0,0,0,1,1,1,0,0],
    ],
    
]

ans = [
    [0, 0, 1, 0], # 2
    [0, 1, 0, 0], # 1
    [1, 0, 0, 0], # 0
    [1, 0, 0, 1], # 3
]



In [ ]:
# 学習部(書きかけ)

class Dataset:
    def __init__(self, images, ans):
        self.images = images
        self.ans = ans

class Model:
    def __init__(self):
        self.w1 = []
        self.w2 = []
        
    def calc(self, image):
        h1 = np.dot(image, self.w1)
        h1 = sigmoid(h1)
        h2 = np.dot(image, self.w2)
        h2 = sigmoid(h2)
        return h2

class NN:
    LEARNING_RATE = 1e-4

    def __init__(self, dataset, model):
        self.w1 = []
        self.w2 = []
        self.dataset = dataset
        self.model = model

    def inference(self, image):
        return self.model.calc(image)
    
    def train(self):
        # deep learning風
        # opt = tf.train.AdamOptimizer(learning_rate)
        for i in range(11111):
            y = self.model.calc(self.dataset.images) # NN本体
            # 評価
            # deep learning風
            # loss = -tf.reduce_sum(ans*tf.log(tf.clip_by_value(y,1e-10,1.0-1e-10)))
            # loss = tf.reduce_sum(ans* -tf.log(y) + (1 - ans) *  -tf.log(1 - y)) # cross entropy
            loss = leastSquares(self.dataset.ans, y) # 最小二乗法等

            # deep learning風
            # 逆関数の微分を使って、進む方向を決める
            # grads = opt.compute_gradients(loss)
            # 進む方向から最適化する
            # train_op = opt.apply_gradients(grads, global_step=global_step)
            # tf.train.AdamOptimizer(最適化)の精度を上げる
            # extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            self.model.w1, self.model.w2 = optimize(LEARNING_RATE, loss) # 最適化

dataset = Dataset(images, ans)
model = Model()
nn = NN(dataset, model)
nn.train()

In [ ]:
# 推論(書きかけ)
ix = 2

def report(y, ans):
    print("test")

report(nn.inference(images[ix]), ans[ix])


### 考察（要検証）



### 閃き方法 (要検証)

- 脳を思いっきりシンプルにして、数学っぽく解けば思いつきそう


### 生活の知恵　（要検証）



# ディープラーニングのコア部分(未実装)

- 特徴抽出の方法として、隣のデータに意味があるする。
- 自動で隣の隣なども拡張できる。
- それらを組み合わせることも自動でできる。
- これをニューラルネットワークの理論で最適化する。

- 概念
    - 高い特徴抽出
    - 超膨大な計算

- ※いろいろあるのでサーバーを提供できません。

[古代のDeepLearningのモデル例](https://github.com/todorokit/tensorflow_cnn_image_sample/blob/master/config-example/v3_modoki.py)

### 組み合わせ例

- Concat # 下記四つ並列
    - Conv2D_bn("conv2-a", 略) # 単品
    - Conv2D_bn("conv2-b-1", 略), Conv2D_bn("conv2-b-2", 略), # 2層直列
    - Conv2D_bn("conv2-c-1", 略), Conv2D_bn("conv2-c-2", 略), Conv2D_bn("conv2-c-3", 略], # 3層直列
    - AveragePooling2D("apool2", 略), Conv2D_bn("conv2-d"], 略)　# AveragePool　含む2層直列


In [ ]:
# 

image = [ # 2
    [0,0,0,0,0,0,0,0],
    [0,0,0,1,1,0,0,0],
    [0,0,1,0,0,1,0,0],
    [0,1,0,0,0,1,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,1,1,1,1,1,0],
]

filter1 = [
    [0,1,0],
    [1,1,1],
    [0,1,0],
]

# h1_1 = image.eachFilter(filter1) # 概念
# image に filter1による部分的な 内積 を全体的に適用して最適化した行列が特徴1となる。

filter2 = [
    [0,1,0],
    [0,1,0],
    [0,1,0],
]

# h1_2 = image.eachFilter(filter2) # 概念
# image に filter2による部分的な 内積 を全体的に適用して最適化した行列が特徴2となる。

# h2 = concat(h1, h2).eachFilter(filter3)
# 組み合わせたものも特徴である。
